In [9]:
import pandas as pd
import os
from datetime import datetime

def analisar_planilha(caminho_arquivo):
    """Analisa a planilha e retorna o resumo por produto em ordem alfabética"""
    try:
        # Carregar o arquivo Excel
        df = pd.read_excel(caminho_arquivo)
        
        # Remover linha de total
        df = df.dropna(subset=['Número Boleto'])
        
        # Calcular totais por produto
        resumo = df.groupby(['Produto', 'Tipo'])['Qtde'].sum().unstack(fill_value=0)
        resumo = resumo.rename(columns={
            'Compra': 'Total Compra',
            'Venda': 'Total Venda'
        })
        
        # Ordenar por produto (ordem alfabética)
        resumo = resumo.sort_index(level='Produto')
        
        # Calcular soma da coluna Total
        soma_total = df['Total'].sum()
        
        return resumo, soma_total
    
    except Exception as e:
        print(f"Erro ao processar o arquivo: {e}")
        return None, None

def salvar_resultados(resumo, pasta_resultados='resultados'):
    """Salva os resultados em formato Excel e CSV com data no nome"""
    # Criar pasta de resultados se não existir
    os.makedirs(pasta_resultados, exist_ok=True)
    
    # Obter data atual para nome do arquivo
    data_atual = datetime.now().strftime("%Y%m%d_%H%M%S")
    nome_base = f"resumo_operacoes_{data_atual}"
    
    # Salvar em Excel
    caminho_excel = os.path.join(pasta_resultados, f"{nome_base}.xlsx")
    resumo.to_excel(caminho_excel)
    
    # Salvar em CSV formatado
    caminho_csv = os.path.join(pasta_resultados, f"{nome_base}.csv")
    resumo.to_csv(caminho_csv)
    
    return caminho_excel, caminho_csv

def exibir_relatorio(resumo, soma_total):
    """Exibe o relatório formatado no console"""
    print("\n" + "="*70)
    print("RELATÓRIO DE OPERAÇÕES")
    print("="*70)
    
    # Formatar números para exibição
    resumo_exibicao = resumo.copy()
    for col in resumo_exibicao.columns:
        resumo_exibicao[col] = resumo_exibicao[col].apply(lambda x: f"{x:,.0f}".replace(",", "."))
    
    # Exibir tabela
    print(resumo_exibicao)
    
    # Exibir soma da coluna R (Total)
    print("\n" + "-"*70)
    print(f"TOTAL: R$ {soma_total:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))
    print("="*70)

# Configurações
PASTA_DOWNLOADS = os.path.expanduser("~/Downloads")  # Pasta onde as planilhas são salvas
PADRAO_ARQUIVO = "Relatório de Operações (*.xls*)"   # Padrão para encontrar arquivos

def main():
    """Função principal do programa"""
    print("="*70)
    print("ANALISADOR DE OPERAÇÕES POR PRODUTO")
    print("="*70)
    
    # Encontrar o arquivo mais recente na pasta de downloads
    arquivos = [f for f in os.listdir(PASTA_DOWNLOADS) 
               if f.startswith("Relatório de Operações") and f.endswith(('.xls', '.xlsx'))]
    
    if not arquivos:
        print("\nERRO: Nenhum arquivo encontrado na pasta de downloads.")
        print(f"Procurei por: {PADRAO_ARQUIVO}")
        print(f"Pasta: {PASTA_DOWNLOADS}")
        return
    
    # Selecionar o arquivo mais recente
    arquivo_mais_recente = max(
        arquivos, 
        key=lambda f: os.path.getmtime(os.path.join(PASTA_DOWNLOADS, f))
    )
    
    caminho_completo = os.path.join(PASTA_DOWNLOADS, arquivo_mais_recente)
    
    print(f"\nArquivo selecionado: {arquivo_mais_recente}")
    print(f"Caminho completo: {caminho_completo}")
    
    # Processar o arquivo
    resumo, soma_total = analisar_planilha(caminho_completo)
    
    if resumo is not None:
        # Salvar resultados
        excel_path, csv_path = salvar_resultados(resumo)
        
        # Exibir relatório
        exibir_relatorio(resumo, soma_total)
        
        print("\nRESULTADOS SALVOS EM:")
        print(f"- Excel: {excel_path}")
        print(f"- CSV:   {csv_path}")
        print("\nPara atualizar, basta baixar uma nova planilha e executar novamente!")

if __name__ == "__main__":
    main()

ANALISADOR DE OPERAÇÕES POR PRODUTO

Arquivo selecionado: Relatório de Operações - 2025-08-22T150949.288.xls
Caminho completo: C:\Users\User/Downloads\Relatório de Operações - 2025-08-22T150949.288.xls
WARNING *** file size (20865) not 512 + multiple of sector size (512)

RELATÓRIO DE OPERAÇÕES
Tipo    Total Compra Total Venda
Produto                         
EUR            3.595       4.645
MNG               90          42
USD              220       7.685

----------------------------------------------------------------------
TOTAL: R$ 100.581,45

RESULTADOS SALVOS EM:
- Excel: resultados\resumo_operacoes_20250822_151003.xlsx
- CSV:   resultados\resumo_operacoes_20250822_151003.csv

Para atualizar, basta baixar uma nova planilha e executar novamente!
